In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate

In [2]:
def reflection_operator(n):
    qc1 = QuantumCircuit(n)
    qc1.x(range(n))
    qc1.h(0)
    qc1.append(MCXGate(n-1), list(range(1, n))+[0])
    qc1.h(0)
    qc1.x(range(n))
    return qc1.to_gate()

In [3]:
def grover_gate(n, oracle_gate):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.append(oracle_gate, range(n))
    qc.h(range(n))
    qc.append(reflection_operator(n), range(n))
    qc.h(range(n))
    return qc.to_gate()

In [4]:
n = 3
N = 2**n

In [5]:
oracle_matrix = np.diag([1, 1, 1, 1, -1, 1, 1, 1])
oracle_gate = UnitaryGate(oracle_matrix)

In [17]:
qc = QuantumCircuit(n)
qc.append(grover_gate(n, oracle_gate), range(n))
qc.append(grover_gate(n, oracle_gate), range(n))
qc.append(grover_gate(n, oracle_gate), range(n))
Statevector(qc)

Statevector([ 3.53553391e-01+0.j, -3.53553391e-01+0.j, -3.53553391e-01+0.j,
             -3.53553391e-01+0.j,  7.07106781e-01+0.j, -1.46090250e-16+0.j,
             -7.31237202e-18+0.j,  2.04432036e-17+0.j],
            dims=(2, 2, 2))


In [6]:
qc.draw()

┌───┐┌──────────┐┌───┐┌─────────────┐┌───┐
q_0: ┤ H ├┤0         ├┤ H ├┤0            ├┤ H ├
     ├───┤│          │├───┤│             │├───┤
q_1: ┤ H ├┤1 Unitary ├┤ H ├┤1 circuit-88 ├┤ H ├
     ├───┤│          │├───┤│             │├───┤
q_2: ┤ H ├┤2         ├┤ H ├┤2            ├┤ H ├
     └───┘└──────────┘└───┘└─────────────┘└───┘

In [8]:
qc.decompose().draw()

global phase: π
     ┌─────────┐┌──────────────┐                                             »
q_0: ┤ U2(0,π) ├┤0             ├──■───────────────────────────────────────■──»
     ├─────────┤│  circuit-144 │  │                                       │  »
q_1: ┤ U2(0,π) ├┤1             ├──┼───────────────────■───────────────────┼──»
     ├─────────┤├─────────────┬┘┌─┴─┐┌─────────────┐┌─┴─┐┌─────────────┐┌─┴─┐»
q_2: ┤ U2(0,π) ├┤ U(0,0,-π/4) ├─┤ X ├┤ U(0,0,-π/4) ├┤ X ├┤ U(0,0,-π/4) ├┤ X ├»
     └─────────┘└─────────────┘ └───┘└─────────────┘└───┘└─────────────┘└───┘»
«                                                                             »
«q_0: ────────────────────────────────────■───────────────────────────────────»
«                                         │                                   »
«q_1: ─────────────────■──────────────────┼────────────────────────────────■──»
«     ┌─────────────┐┌─┴─┐┌────────────┐┌─┴─┐┌────────────┐┌────────────┐┌─┴─┐»
«q_2: ┤ U(0,0,-π/4) ├┤ X ├┤ U(π/2,0,π) ├┤ X ├┤ U(π/2,0,π) ├┤ U(π/2,0,π) ├┤ X ├»
«     └─────────────┘└───┘└────────────┘└───┘└────────────┘└────────────┘└───┘»
«                                      ┌──────────────┐                    »
«q_0: ──────────────────────────────■──┤0             ├─────────────────■──»
«                                   │  │  circuit-151 │                 │  »
«q_1: ──────────────────────────────┼──┤1             ├─────────────────┼──»
«     ┌────────────┐┌────────────┐┌─┴─┐└┬────────────┬┘┌─────────────┐┌─┴─┐»
«q_2: ┤ U(π/2,0,π) ├┤ U(π/2,0,π) ├┤ X ├─┤ U(π/2,0,π) ├─┤ U(0,0,-π/2) ├┤ X ├»
«     └────────────┘└────────────┘└───┘ └────────────┘ └─────────────┘└───┘»
«                                   ┌──────────────┐┌─────────┐┌───┐┌───┐┌───┐»
«q_0: ───────■──────────────────────┤0             ├┤ U2(0,π) ├┤ X ├┤ H ├┤ X ├»
«            │                      │  circuit-160 │├─────────┤├───┤└───┘└─┬─┘»
«q_1: ──■────┼───────────────────■──┤1             ├┤ U2(0,π) ├┤ X ├───────■──»
«     ┌─┴─┐┌─┴─┐┌─────────────┐┌─┴─┐└─┬─────────┬──┘└──┬───┬──┘└───┘       │  »
«q_2: ┤ X ├┤ X ├┤ U(0,0,-π/2) ├┤ X ├──┤ U2(0,π) ├──────┤ X ├───────────────■──»
«     └───┘└───┘└─────────────┘└───┘  └─────────┘      └───┘                  »
«     ┌───┐   ┌───┐   ┌─────────┐
«q_0: ┤ H ├───┤ X ├───┤ U2(0,π) ├
«     ├───┤┌──┴───┴──┐└─────────┘
«q_1: ┤ X ├┤ U2(0,π) ├───────────
«     ├───┤├─────────┤           
«q_2: ┤ X ├┤ U2(0,π) ├───────────
«     └───┘└─────────┘

In [9]:
# print(qc.qasm())

OPENQASM 2.0;
include "qelib1.inc";
gate circuit_160 q0,q1 { u3(0,-1.0766785893143451,-0.49411773748055143) q0; u3(2.300511030367451,-pi/2,0) q1; cx q0,q1; u3(pi,-3*pi/4,3*pi/4) q0; u3(1.4247381821882348,-1.0089795545550775,2.0674326619635908) q1; cx q0,q1; u3(pi,2.1371438478931815,-1.78984696909406) q0; u3(2.4118779500172396,-pi,pi/2) q1; }
gate circuit_151 q0,q1 { u3(0,-pi,-pi) q0; u3(pi/2,-pi,0.8410816232223421) q1; cx q0,q1; u3(0,-0.16991845472706046,1.740714781521957) q0; u3(pi/2,-2.30051103036745,-pi) q1; }
gate circuit_144 q0,q1 { u3(pi,1.4065829705916304,-1.7350096829981632) q0; u3(pi/2,-pi,0.8410816232223421) q1; cx q0,q1; u3(pi,0.16991845472706046,-1.4008778720678343) q0; u3(pi/2,-1.6264797866197904,0) q1; }
gate unitary140167290891808 p0,p1,p2 {
	circuit-144 p0,p1;
	u(0,0,-pi/4) p2;
	cx p0,p2;
	u(0,0,-pi/4) p2;
	cx p1,p2;
	u(0,0,-pi/4) p2;
	cx p0,p2;
	u(0,0,-pi/4) p2;
	cx p1,p2;
	u(pi/2,0,pi) p2;
	cx p0,p2;
	u(pi/2,0,pi) p2;
	u(pi/2,0,pi) p2;
	cx p1,p2;
	u(pi/2,0,pi) p2;
	u(